In [1]:
!pip install pycaret

In [2]:
import sys
print(sys.version)
from pycaret.classification import *

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [3]:
import pandas as pd

In [4]:
# 1) Load data
df = pd.read_csv("ckd.csv")
print("Original shape:", df.shape)
display(df.head())

Original shape: (1659, 54)


,PatientID,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,...,Itching,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis,DoctorInCharge
0,1,71,0,0,0,2,31.069414,1,5.128112,1.676220,...,7.556302,76.076800,0,0,1,1.018824,4.966808,9.871449,1,Confidential
1,2,34,0,0,1,3,29.692119,1,18.609552,8.377574,...,6.836766,40.128498,0,0,0,3.923538,8.189275,7.161765,1,Confidential
2,3,80,1,1,0,1,37.394822,1,11.882429,9.607401,...,2.144722,92.872842,0,1,1,1.429906,7.624028,7.354632,1,Confidential
3,4,40,0,2,0,1,31.329680,0,16.020165,0.408871,...,7.077188,90.080321,0,0,0,3.226416,3.282688,6.629587,1,Confidential
4,5,43,0,1,1,2,23.726311,0,7.944146,0.780319,...,3.553118,5.258372,0,0,1,0.285466,3.849498,1.437385,1,Confidential


In [5]:
# 2) Drop unnecessary columns
drop_cols = [c for c in ["PatientID", "DoctorInCharge"] if c in df.columns]
df = df.drop(columns=drop_cols)
print("After drop shape:", df.shape)
print("Dropped:", drop_cols)

After drop shape: (1659, 52)
Dropped: ['PatientID', 'DoctorInCharge']


In [6]:
# 3) Check class distribution
print("\nDiagnosis distribution:")
print(df["Diagnosis"].value_counts(dropna=False))


Diagnosis distribution:
Diagnosis
1    1524
0     135
Name: count, dtype: int64


In [7]:
# 4) Setup
clf = setup(
    data=df,
    target="Diagnosis",
    session_id=42,
    train_size=0.8,
    fold=5,
    # preprocessing
    normalize=True,
    fix_imbalance=True, 
    remove_multicollinearity=True,
    multicollinearity_threshold=0.95,
    verbose=True
)

,Description,Value
0,Session id,42
1,Target,Diagnosis
2,Target type,Binary
3,Original data shape,"(1659, 52)"
4,Transformed data shape,"(2770, 52)"
5,Transformed train set shape,"(2438, 52)"
6,Transformed test set shape,"(332, 52)"
7,Numeric features,51
8,Preprocess,True
9,Imputation type,simple


In [8]:
# 5) Compare models (choose best by F1)
best_model = compare_models(sort="F1", n_select=1)
print("\nBest model:", best_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9322,0.8313,0.9943,0.9359,0.9642,0.3317,0.4027,0.5260
xgboost,Extreme Gradient Boosting,0.9299,0.8359,0.9877,0.9392,0.9628,0.3614,0.4108,0.3000
rf,Random Forest Classifier,0.9224,0.7949,0.9975,0.9240,0.9594,0.1207,0.2202,0.2700
gbc,Gradient Boosting Classifier,0.9224,0.8309,0.9852,0.9339,0.9589,0.2735,0.3089,0.8680
et,Extra Trees Classifier,0.9209,0.7698,1.0000,0.9207,0.9587,0.0498,0.1248,0.1360
qda,Quadratic Discriminant Analysis,0.9194,0.5833,1.0000,0.9193,0.9580,0.0169,0.0420,0.0720
ada,Ada Boost Classifier,0.8975,0.7968,0.9418,0.9465,0.9441,0.3289,0.3306,0.2100
dt,Decision Tree Classifier,0.8432,0.6274,0.8851,0.9408,0.9120,0.1967,0.2051,0.1800
nb,Naive Bayes,0.8350,0.6896,0.8843,0.9326,0.9076,0.1293,0.1342,2.6480
lr,Logistic Regression,0.7898,0.8009,0.8023,0.9628,0.8750,0.2428,0.2915,2.8700



Best model: LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [9]:
# 6) Hyperparameter tuning
tuned_model = tune_model(best_model, optimize="F1")
print("\nTuned model:", tuned_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9248,0.8465,0.9918,0.9308,0.9603,0.2595,0.3221
1,0.9361,0.7975,0.9959,0.9382,0.9662,0.3894,0.4622
2,0.9132,0.8249,0.9713,0.9368,0.9537,0.2604,0.2720
3,0.9547,0.9403,0.9959,0.9567,0.9759,0.6034,0.6393
4,0.9245,0.7806,0.9918,0.9305,0.9602,0.2594,0.3220
Mean,0.9307,0.8380,0.9893,0.9386,0.9633,0.3544,0.4035
Std,0.0140,0.0559,0.0092,0.0096,0.0075,0.1342,0.1339


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).

Tuned model: LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [10]:
!pip install -U ipywidgets

In [11]:
import ipywidgets as widgets
widgets

<module 'ipywidgets' from 'E:\\AutoML\\pycaret_env\\Lib\\site-packages\\ipywidgets\\__init__.py'>

In [12]:
# 7) Evaluate (interactive)
from IPython.display import display
display(evaluate_model(tuned_model))

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

None

In [13]:
# 8) Predictions on holdout
pred_holdout = predict_model(tuned_model)
display(pred_holdout.head())

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9127,0.7592,0.9803,0.9286,0.9537,0.1802,0.2055


,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis,prediction_label,prediction_score
1232,66,1,1,1,1,32.441158,0,10.534957,0.323453,7.338599,...,40.090206,0,0,0,0.650166,8.744802,7.338390,1,1,0.9998
1600,58,0,0,1,1,22.454872,0,12.426801,5.691748,8.541965,...,55.169525,0,0,0,2.911090,1.838819,2.420768,0,1,0.9958
152,31,1,1,1,1,36.059284,0,5.044001,6.656737,1.467807,...,58.353710,0,0,0,0.787877,6.365757,4.156352,1,1,0.9950
491,90,1,2,0,2,22.340809,1,7.060485,2.986460,1.334690,...,53.348938,0,0,0,2.752223,4.985360,6.824299,1,1,0.9989
412,32,0,1,2,2,23.505405,0,14.124260,8.877092,9.825925,...,46.650028,0,0,0,0.076519,4.752793,2.079430,1,1,0.9957


In [14]:
# 9) Metrics table (last pulled results)
results = pull()
display(results)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9127,0.7592,0.9803,0.9286,0.9537,0.1802,0.2055
